In [96]:
import s3fs
import json
import gzip
from collections import defaultdict

In [41]:
class S3(s3fs.S3FileSystem):
    def lsr(fs, path):
        filenames = []
        def go(path):
            paths = fs.ls(path, detail=True)
            for path in paths:
                if path['StorageClass'] == 'DIRECTORY':
                    go(path['Key'])
                else:
                    filenames.append(path['Key'])
        go(path)
        return filenames
    
    def jsongz(fs, path):
        with fs.open(path, 'rb') as f:
            with gzip.GzipFile(fileobj=f) as gz:
                while True:
                    line = gz.readline()
                    if not line:
                        return
                    else:
                        yield json.loads(line)

In [101]:
fs = S3(key='minio',
        secret='miniostorage',
        use_ssl=False,
        client_kwargs={'endpoint_url': 'http://localhost:5000'})

bucket = 'localhost'

counts = defaultdict(int)
for filename in fs.lsr(bucket + '/secor/json'):
    for line in fs.jsongz(filename):
        counts[filename] += 1

for f, c in counts.iteritems():
    print f, c

localhost/secor/json/dt=2017-06-28/1_0_00000000000000006240.gz 520
localhost/secor/json/dt=2017-06-28/1_0_00000000000000005720.gz 520


In [107]:
import datetime, dateutil, random

In [103]:
baseline = int(dateutil.parser.parse('2017-06-28T15:59:53.849Z').strftime('%s'))

In [108]:
with open('crap.txt', 'w') as f:
    for sec in xrange(60*60*2):
        data = json.dumps([str(random.random()) for _ in xrange(10)])
        timestamp = datetime.datetime.fromtimestamp(baseline+sec).isoformat() + '.000Z'
        f.write(json.dumps(dict(timestamp=timestamp, data=data)))
        f.write('\n')